# Library

In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# Dataset

In [3]:
path_train = "C:/Users/julia/Downloads/forecasting/train.csv"
path_test = "C:/Users/julia/Downloads/forecasting/test.csv"
path_train_calendar = "C:/Users/julia/Downloads/forecasting/train_calendar.csv"
path_test_calendar = "C:/Users/julia/Downloads/forecasting/test_calendar.csv"
# train dataset 
train = pd.read_csv(path_train)
train_calendar = pd.read_csv(path_train_calendar)
test = pd.read_csv(path_test)
test_calendar = pd.read_csv(path_test_calendar)

### Train_calendar and Test_calendar Explaination
    train_calendar.csv - a calendar for the training set containing data about holidays or warehouse specific events, some columns are already in the train data but there are additional rows in this file for dates where some warehouses could be closed due to public holiday or Sunday (and therefore they are not in the train set)

### Dataset explaination

    1.warehouse - warehouse name
    2.date - date
    3.orders - number of customer orders attributed to the warehouse
    4.holiday_name - name of public holiday if any
    5.holiday - 0/1 indicating the presence of holidays
    6.shutdown - warehouse shutdown or limitation due to operations (not provided in test)
    7.mini_shutdown - warehouse shutdown or limitation due to operations (not provided in test)
    8.shops_closed - public holiday with most of the shops or large part of shops closed
    9.winter_school_holidays - school holidays
    10.school_holidays - school holidays
    11.blackout - warehouse shutdown or limitation due to operations (not provided in test)
    12.mov_change - a change in minimum order value indicating potential change in customer behaviour (not provided in test)
    13.frankfurt_shutdown - warehouse shutdown or limitation due to operations (not provided in test)
    14.precipitation - precipitation in mm around the location of the warehouse which correlates with location of the customers (not provided in test)
    15.snow - snowfall in mm around the location of the warehouse which correlates with location of the customers (not provided in test)
    16.user_activity_1 - user activity on the website (not provided in test)
    17.user_activity_2 - user activity on the website (not provided in test)
    18.id - row id consisting of warehouse name and date


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7340 entries, 0 to 7339
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   warehouse               7340 non-null   object 
 1   date                    7340 non-null   object 
 2   orders                  7340 non-null   float64
 3   holiday_name            218 non-null    object 
 4   holiday                 7340 non-null   int64  
 5   shutdown                7340 non-null   int64  
 6   mini_shutdown           7340 non-null   int64  
 7   shops_closed            7340 non-null   int64  
 8   winter_school_holidays  7340 non-null   int64  
 9   school_holidays         7340 non-null   int64  
 10  blackout                7340 non-null   int64  
 11  mov_change              7340 non-null   float64
 12  frankfurt_shutdown      7340 non-null   int64  
 13  precipitation           7070 non-null   float64
 14  snow                    7070 non-null   

In [6]:
train_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307 entries, 0 to 13306
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    13307 non-null  object 
 1   holiday_name            563 non-null    object 
 2   holiday                 13307 non-null  int64  
 3   shutdown                13307 non-null  int64  
 4   mini_shutdown           13307 non-null  int64  
 5   warehouse_limited       13307 non-null  int64  
 6   shops_closed            13307 non-null  int64  
 7   winter_school_holidays  13307 non-null  int64  
 8   school_holidays         13307 non-null  int64  
 9   blackout                13307 non-null  int64  
 10  mov_change              13307 non-null  float64
 11  frankfurt_shutdown      13307 non-null  int64  
 12  precipitation           11199 non-null  float64
 13  snow                    11199 non-null  float64
 14  warehouse               13307 non-null

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   warehouse               397 non-null    object
 1   date                    397 non-null    object
 2   holiday_name            17 non-null     object
 3   holiday                 397 non-null    int64 
 4   shops_closed            397 non-null    int64 
 5   winter_school_holidays  397 non-null    int64 
 6   school_holidays         397 non-null    int64 
 7   id                      397 non-null    object
dtypes: int64(4), object(4)
memory usage: 24.9+ KB


In [8]:
test_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1869 entries, 0 to 1868
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    1869 non-null   object
 1   holiday_name            69 non-null     object
 2   holiday                 1869 non-null   int64 
 3   shops_closed            1869 non-null   int64 
 4   winter_school_holidays  1869 non-null   int64 
 5   school_holidays         1869 non-null   int64 
 6   warehouse               1869 non-null   object
dtypes: int64(4), object(3)
memory usage: 102.3+ KB


#### NO duplicate data in train and train_calendar dataset

In [19]:
# check wether train,train_calender has duplicate data
print(train[train.duplicated()])
print(train_calendar[train_calendar.duplicated()])

Empty DataFrame
Columns: [warehouse, date, orders, holiday_name, holiday, shutdown, mini_shutdown, shops_closed, winter_school_holidays, school_holidays, blackout, mov_change, frankfurt_shutdown, precipitation, snow, user_activity_1, user_activity_2, id]
Index: []
Empty DataFrame
Columns: [date, holiday_name, holiday, shutdown, mini_shutdown, warehouse_limited, shops_closed, winter_school_holidays, school_holidays, blackout, mov_change, frankfurt_shutdown, precipitation, snow, warehouse]
Index: []


#### Train dataset don't have warehouse_limited, train_calendar don't have oders(critical target), website_activity_1,website_activity_2, and id

#### Train dataset id is actually warehouse_date, so before join train and train_calendar, create id in train_calendar (join on id)

In [40]:
# check wether id = warehouse + date in train
warehouse_date = train['warehouse'] + '_' + train['date']
torf = warehouse_date == train['id']
torf.value_counts()

True    7340
Name: count, dtype: int64

In [62]:
# create a new columns id in train_calendar
id = train_calendar['warehouse'] + '_' + train_calendar['date']
train_calendar['id'] = id

#### New dataset for train_calendar1(it has critical target :orders, it also has website_activity_1, website_activity_2, and id )

In [108]:
# join the dataset fully join on id
train_calendar1 = pd.merge(train, train_calendar, how = 'outer', on = 'id', suffixes= ('_remove',''))
train_calendar1.drop([i for i in train_calendar1.columns if '_remove' in i], axis = 1, inplace = True)
train_calendar1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307 entries, 0 to 13306
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   orders                  7340 non-null   float64
 1   user_activity_1         7340 non-null   float64
 2   user_activity_2         7340 non-null   float64
 3   id                      13307 non-null  object 
 4   date                    13307 non-null  object 
 5   holiday_name            563 non-null    object 
 6   holiday                 13307 non-null  int64  
 7   shutdown                13307 non-null  int64  
 8   mini_shutdown           13307 non-null  int64  
 9   warehouse_limited       13307 non-null  int64  
 10  shops_closed            13307 non-null  int64  
 11  winter_school_holidays  13307 non-null  int64  
 12  school_holidays         13307 non-null  int64  
 13  blackout                13307 non-null  int64  
 14  mov_change              13307 non-null

#### edit new dataset in train_calendar1(remove orders which is empty), and change it to name: train_1( train_1 has new columns: warehouse_limited)

In [122]:
condition = train_calendar1['orders'].isnull()
train_1 = train_calendar1[~condition]

#### Add day, month, and year in new dataset train_1

In [121]:
# transform date string to date format
date = [datetime.strptime(i, '%Y-%m-%d')for i in train_1['date']]
# extract year, month, day in date
year = [i.year for i in date]
month = [i.month for i in date]
day = [i.day for i in date]
# add year , month , day to train_1 
train_1.loc[:,'year'] = year
train_1.loc[:,'month'] = month
train_1.loc[:,'day'] = day

In [123]:
train_1['warehouse'].value_counts()

warehouse
Brno_1         1193
Prague_1       1193
Prague_2       1193
Prague_3       1193
Budapest_1     1154
Munich_1        785
Frankfurt_1     629
Name: count, dtype: int64

#### build train dataset for different warehouse

In [129]:
Brno1_train = train_1[train_1['warehouse'] == 'Brno_1']
Prague1_train = train_1[train_1['warehouse'] == 'Prague_1']
Prague2_train = train_1[train_1['warehouse'] == 'Prague_2']
Prague3_train = train_1[train_1['warehouse'] == 'Prague_3']
Budapest1_train = train_1[train_1['warehouse'] == 'Budapest_1']
Munich1_train = train_1[train_1['warehouse'] == 'Munich_1']
Frankfurt1_train = train_1[train_1['warehouse'] == 'Frankurt_1']
